# Regresión lineal: online

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#--para comparar nuestros resultados con la solución exacta
from sklearn.linear_model import LinearRegression

#--para escalar los datos
from sklearn.preprocessing import StandardScaler

#--para utilizar SGD de sklearn
from sklearn.linear_model import SGDRegressor

El objetivo es realizar una regresión lineal para explicar los niveles de ozono dada la temperatura utilizando el dataset "airquality.csv".

Compararemos el resultado de la regresión lineal en línea con la solución "exacta" de scikit-learn. También exploraremos la función de regresión lineal basada en gradiente estocástico de scikit-learn.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#--cargamos todo el data set en memoria
database_file = '/content/drive/MyDrive/data_sets/airquality.csv'
dat = pd.read_csv(database_file)
dat.head()

In [ ]:
#--para utilizar gradient descent es necesario que los atributos tengan
#-las mismas escalas. Aunque en este ejemplo solo estamos utilizando
#-una variable por lo que la normalización no toma relevancia.

#--definimos una instancia para normalizar los datos
escalar = StandardScaler()

#--normalizamos los datos
dat_scaled = escalar.fit_transform(dat[['Temp']])

print(dat_scaled[:5])

La variable respuesta no necesita ser normalizada

In [ ]:
#--utilizando el dataset completo en memoria, calculamos la solución exacta
#-para después compararla con los métodos en línea.

#--definimos una instancia de regresión lineal
modelo_exacto = LinearRegression()
#--entrenamos el modelo
modelo_exacto.fit(X = dat_scaled, y = dat['Ozone'])
#--observamos sus parámetros o coeficientes
print(f'theta_0: {np.round(modelo_exacto.intercept_,2)}')
print(f'theta_1: {np.round(modelo_exacto.coef_[0])}')

#--graficamos el resultado
#-no dibujo todos los puntos, solo me interesa saber el inicio y el final
#-para unirlos por una línea
minimo = dat_scaled.min()
maximo = dat_scaled.max()
new_sample = np.array([[minimo], [maximo]])
y_hat_exacto = modelo_exacto.predict(new_sample)

plt.plot(dat_scaled, dat['Ozone'], 'ko')
plt.plot(new_sample, y_hat_exacto, 'b-')
plt.title('Modelo exacto')

plt.show();

Solución usando batch gradient descent (se usa todo el data set para calcular una actualización a los parámetros).

Asumimos que tenemos acceso a todo el dataset en memoria.

In [ ]:
alpha = .0001    #--learning rate
epochs = 300    #--cuantas veces iteramos sobre TODO el dataset

#--variable respuesta
y = dat[['Ozone']]
#--numero de muestras
m = y.shape[0]
#--agregamos una columna de unos a los atributos
X = np.ones((m, 2))
X[:,1:] = dat_scaled
#--inicializamos las thetas a cero
thetas = np.zeros((2,1))
#--iteramos el número de epocas definido previamente
for i in range(epochs):
    #--calculamos la predicción con las thetas actuales
    y_hat = np.dot(X, thetas)
    #--obtenemos el error
    error = y_hat - y
    #--actualizamos las thetas
    thetas = thetas - alpha * np.dot(X.T,error)

#--creamos una muestra nueva con una columna de unos
new_sample_ones = np.ones((2,2))
new_sample_ones[:,1:] = new_sample
#--hacemos la predicción
y_hat = np.dot(new_sample_ones, thetas)
#--ploteamos los resultados
plt.plot(dat_scaled, dat['Ozone'], 'ko')
plt.plot(new_sample, y_hat_exacto, 'b-', label='exacto')
plt.plot(new_sample, y_hat, 'r--', label='gradient descent')
plt.legend()
plt.show();

Utilizaremos __stochastic gradient descent__ (actualizamos los parámetros con cada muestra). Aún consideramos que tenemos el dataset completo en memoria.

In [ ]:
alpha = .001    #--learning rate
epochs = 1000     #--cuantas veces iteramos sobre TODO el dataset

#--variable respuesta
y = dat['Ozone']
#--número de muestras
m = y.shape[0]
#--agregamos una columna de unos al training set
X = np.ones((m, 2))
X[:,1:] = dat_scaled
#--definos las thetas como un vector de ceros
thetas = np.zeros((2,1))
#--iteramos sobre el número de epocas
for i in range(epochs):
    #--iteramos sobre cada muestra del training set
    for j in range(m):
        #--calculamos la predicción con las thetas actuales
        y_hat = np.dot(X[None,j,:], thetas)
        #--observamos el error
        error = y_hat - y[j]
        #--actualizamos las thetas. Observen que empezamos a
        #-actuilzar las thetas con la primera muesta que vemos,
        #-a diferencia de batch gradient descent que realiza
        #-su primera actualización, una vez que vio todo el dataset
        thetas = thetas - alpha * np.dot(X[None,j,:].T, error)


#--hacemos la predicción
y_hat = np.dot(new_sample_ones, thetas)
#--ploteamos los resultados
plt.plot(dat_scaled, dat['Ozone'], 'bo')
plt.plot(new_sample, y_hat_exacto, 'k-', label='exacto')
plt.plot(new_sample, y_hat, 'r--', label='stochastic gradient descent')
plt.legend()
plt.show();

Scikit-learn tiene una función que realiza la regresión lineal usando sotchastic gradient descent

In [ ]:
alpha = .01   #--learning rate
epochs = 30    #--cuantas veces iteramos sobre TODO el dataset

m = dat_scaled.shape[0]
modelo_sklearn = SGDRegressor(eta0=alpha)
for i in range(epochs):
    for j in range(m):
        modelo_sklearn.partial_fit(X = dat_scaled[None, j, :],
                                   y = dat.loc[j,['Ozone']])

new_sample = np.array([[minimo], [maximo]])
y_hat_sklearn = modelo_sklearn.predict(new_sample)

plt.plot(dat_scaled, dat['Ozone'], 'bo')
plt.plot(new_sample, y_hat_exacto, 'k-')
plt.plot(new_sample, y_hat_sklearn, 'r--');

Finalmente, consideremos el caso donde no cargamos todo el dataset en memoria y utilizamos "stochastic gradient descent".

Debemos conocer la media y la varianza de cada atributo para normalizar los datos.
* Podemos leer una vez el data set y calcular los parametros necesarios.
* Si previamente obtuvimos una muestra, la podemos utilizar para obtener los parámetros que necesitamos.

En este ejemplo, vamos a leer el data set para calcular el promedio y la desviación estandar.

In [ ]:
#--calculo del promedio y desviación estandar

n_samples = 0
sum_x = 0
sum_x2 = 0

reader = pd.read_csv(database_file, iterator = True)
try:
    #--iteramos hasta el final
    while reader:
        row = reader.get_chunk(1)
        sum_x += row.values[0,3] #--la columna 3 es Temp
        sum_x2 += row.values[0,3]**2
        n_samples += 1
except StopIteration:
    pass
finally:
    del reader

promedio = sum_x / n_samples
var = sum_x2/n_samples - promedio**2
std = var**(1/2)
print(f'promedio: {np.round(promedio,2)}')
print(f'desviación estandard: {np.round(std,2)}')

Finalmente calculamos la regresión lineal con los parámetros anteriores y leyendo el archivo poco a poco.

In [ ]:
alpha = .001    #--learning rate
epochs = 30     #--cuantas veces iteramos sobre TODO el dataset


#--definos las thetas como un vector de ceros
thetas = np.zeros((2,1))
#--iteramos sobre el número de epocas
for i in range(epochs):
    #--iteramos sobre cada muestra del training set
    reader = pd.read_csv(database_file, iterator = True)
    try:
        #--iteramos hasta el final
        while reader:
            row = reader.get_chunk(1)
            temp = (row.values[0,3] - promedio) / std
            X = np.array([[1,temp]])
            y_hat = np.dot(X, thetas)
            #--observamos el error
            error = y_hat - row.values[0,0] #--la columna 0 es Ozone
            thetas = thetas - alpha * np.dot(X.T, error)
    except StopIteration:
        pass
    finally:
        del reader


#--hacemos la predicción
y_hat = np.dot(new_sample_ones, thetas)
#--ploteamos los resultados
plt.plot(dat_scaled, dat['Ozone'], 'bo')
plt.plot(new_sample, y_hat_exacto, 'k-', label='exacto')
plt.plot(new_sample, y_hat, 'r--', label='stochastic gradient descent')
plt.legend()
plt.show();